In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn as sk

# Load data
(train_images, train_labels), (test_images, test_labels) = (
    tf.keras.datasets.cifar10.load_data()
)

class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

# Plot model performance
def plot_history(history):
    train_loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    train_acc = history.history["acc"]
    val_acc = history.history["val_acc"]

    plt.figure(figsize=(10, 10))

    plt.subplot(121)
    plt.plot(np.arange(1, len(train_loss) + 1), train_loss)
    plt.plot(np.arange(1, len(train_loss) + 1), val_loss)
    plt.legend(["train", "validation"])
    plt.grid()
    plt.xlabel("epoch")
    plt.ylabel("loss")

    plt.subplot(122)
    plt.plot(np.arange(1, len(train_loss) + 1), train_acc)
    plt.plot(np.arange(1, len(train_loss) + 1), val_acc)
    plt.legend(["train", "validation"])
    plt.grid()
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    
    # Define custom DenseBlock without bottleneck layer
class DenseBlock(tf.keras.layers.Layer):
    def __init__(self, num_layers, growth_rate, bottleneck=False):
        super(DenseBlock, self).__init__()

        self.num_layers = num_layers
        self.growth_rate = growth_rate
        self.bottleneck = bottleneck

        self.layers_list = []
        for _ in range(num_layers):
            # Create a layer composition function
            layer_composition = []
            if bottleneck:
                layer_composition.append(
                    tf.keras.layers.Conv2D(
                        4 * growth_rate,
                        kernel_size=(1, 1),
                        padding="same",
                        kernel_initializer="he_uniform",
                    )
                )
                layer_composition.append(tf.keras.layers.BatchNormalization())
                layer_composition.append(tf.keras.layers.ReLU())
            layer_composition.extend(
                [
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),
                    tf.keras.layers.Conv2D(
                        growth_rate,
                        kernel_size=(3, 3),
                        padding="same",
                        kernel_initializer="he_uniform",
                    ),
                ]
            )
            self.layers_list.append(layer_composition)

        self.concat_layer = tf.keras.layers.Concatenate()

    def call(self, x):
        inputs = [x]  # store the feature maps

        for i in range(self.num_layers):
            layer_composition = self.layers_list[i]  # get current layer composition

            y = x

            for layer in layer_composition:
                y = layer(y)

            # Add to inputs list
            inputs.append(y)

            # concat - feature reuse
            x = self.concat_layer(inputs)

        return x
    
# Transition Layer
class TransitionLayer(tf.keras.layers.Layer):
    def __init__(self, compression_factor):
        super(TransitionLayer, self).__init__()

        self.compression_factor = compression_factor  # compression_factor factor

        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()
        self.avg_pool = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2)

    def build(self, input_shape):
        channels = int(self.compression_factor * input_shape[-1])
        self.conv = tf.keras.layers.Conv2D(channels, kernel_size=(1, 1), padding="same")
        super(TransitionLayer, self).build(input_shape)

    def call(self, x):
        x = self.batch_norm(x)
        x = self.relu(x)
        x = self.conv(x)
        out = self.avg_pool(x)
        return out

In [3]:
GROWTH_RATE = 12
COMPRESSION_FACTOR = 0.5
DROPOUT_RATE = 0.2
EPOCHS = 100

# Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=0.1,
    decay_steps=EPOCHS * len(train_images),
)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    ]
)

# Define model architecture
in_layer = tf.keras.layers.Input(shape=(32, 32, 3))  # input layer
x = data_augmentation(in_layer)  # data augmentation

x = tf.keras.layers.Rescaling(1.0 / 255)(x)  # normalization layer

b = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding="same")(x)

b = DenseBlock(6, GROWTH_RATE, bottleneck=True)(b)
b = tf.keras.layers.Dropout(DROPOUT_RATE)(b)
b = TransitionLayer(compression_factor=COMPRESSION_FACTOR)(b)

b = DenseBlock(12, GROWTH_RATE, bottleneck=True)(b)
b = tf.keras.layers.Dropout(DROPOUT_RATE)(b)
b = TransitionLayer(compression_factor=COMPRESSION_FACTOR)(b)

b = DenseBlock(24, GROWTH_RATE, bottleneck=True)(b)
b = tf.keras.layers.Dropout(DROPOUT_RATE)(b)
b = TransitionLayer(compression_factor=COMPRESSION_FACTOR)(b)

b = DenseBlock(12, GROWTH_RATE, bottleneck=True)(b)
b = tf.keras.layers.Dropout(DROPOUT_RATE)(b)
b = TransitionLayer(compression_factor=COMPRESSION_FACTOR)(b)

b = tf.keras.layers.GlobalAveragePooling2D()(b)  # Global Average Pooling
out_layer = tf.keras.layers.Dense(10, activation="softmax")(b)

# Build model
model = tf.keras.Model(inputs=in_layer, outputs=out_layer)

model.summary()

# Compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])

# Train model
history = model.fit(
    train_images, train_labels,
    epochs=100,
    batch_size=64,
    validation_split=0.1,
    callbacks=tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True),
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_block (DenseBlock)        │ (None, 32, 32, 104)    │        51,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 104)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transition_layer                │ (None, 16, 16, 52)     │         5,876 │
│ (TransitionLayer)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_block_1 (DenseBlock)      │ (None, 16, 16, 196)    │       135,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 16, 196)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transition_layer_1              │ (None, 8, 8, 98)       │        20,090 │
│ (TransitionLayer)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_block_2 (DenseBlock)      │ (None, 8, 8, 386)      │       406,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 386)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transition_layer_2              │ (None, 4, 4, 193)      │        76,235 │
│ (TransitionLayer)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_block_3 (DenseBlock)      │ (None, 4, 4, 337)      │       216,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 4, 4, 337)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transition_layer_3              │ (None, 2, 2, 168)      │        58,132 │
│ (TransitionLayer)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 168)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 973,711 (3.71 MB)

 Trainable params: 961,297 (3.67 MB)

 Non-trainable params: 12,414 (48.49 KB)

Epoch 1/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 578s 780ms/step - acc: 0.3601 - loss: 1.7416 - val_acc: 0.5228 - val_loss: 1.4288
Epoch 2/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 515s 731ms/step - acc: 0.6086 - loss: 1.0959 - val_acc: 0.5984 - val_loss: 1.1878
Epoch 3/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 498s 707ms/step - acc: 0.6948 - loss: 0.8577 - val_acc: 0.6650 - val_loss: 1.0210
Epoch 4/100
526/704 ━━━━━━━━━━━━━━━━━━━━ 2:03 694ms/step - acc: 0.7485 - loss: 0.7091

KeyboardInterrupt: 